# Get Pythonic with the `collections` module

## First day: your new data structure friends

> The collections module implements specialized container datatypes providing alternatives to Python’s general purpose built-in containers, dict, list, set, and tuple. - [docs](https://docs.python.org/3.7/library/collections.html)

In this Day 1 lesson we will use the most common `collections` types showing some easy to follow examples. Day 2 we will get some more practice using them on a movie data set. Day 3 we have you further practice using your own data. I am using Python 3.6 for this notebook.

In [ ]:
from collections import defaultdict, namedtuple, Counter, deque
import csv
import random
from urllib.request import urlretrieve

### 1. `namedtuple`

A `namedtuple` is a convenient way to define a class without methods. This allows you to store `dict` like objects you can access by attributes. Let's first look at a classic `tuple`:

In [ ]:
user = ('bob', 'coder')

The order is not really meaningful leading to ugly code to output the data:

In [ ]:
f'{user[0]} is a {user[1]}'

Let's contrast that with a `namedtuple`:

In [ ]:
User = namedtuple('User', 'name role')

You can directly see that the object has a name and role:

In [ ]:
user = User(name='bob', role='coder')

In [ ]:
user.name

In [ ]:
user.role

Making last string much more informational and elegant (f-strings helps too of course - now you know why we use Python >= 3.6)

In [ ]:
f'{user.name} is a {user.role}'

Conclusion: use a `namedtuple` wherever you can! They are easy to implement and make your code more readable.

### 2. `defaultdict`

I guess you are all too familiar with `KeyError` when using a `dict`, no? 

In [ ]:
users = {'bob': 'coder'}

In [ ]:
users['bob']
users['julian']  # oops

You can get around it with dict's get method:

In [ ]:
users.get('bob')

In [ ]:
users.get('julian') is None

But what if you need to build up a collection though? Let's make a dict from the following list of tuples:

In [ ]:
challenges_done = [('mike', 10), ('julian', 7), ('bob', 5),
                   ('mike', 11), ('julian', 8), ('bob', 6)]
challenges_done

In [ ]:
challenges = {}
for name, challenge in challenges_done:
    #print(name, challenge)
    challenges[name].append(challenge)

In [ ]:
challenges = defaultdict(list)
for name, challenge in challenges_done:
    challenges[name].append(challenge)

challenges

### 3. `Counter`

One of my favorites. Say you want to count the most common words in a text:

In [ ]:
words = """Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been 
the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and 
scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into 
electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of
Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus
PageMaker including versions of Lorem Ipsum""".split()
words[:5]

Before getting to know `collections` I would has written something like this:

In [ ]:
common_words = {}

for word in words:
    if word not in common_words:
        common_words[word] = 0
    common_words[word] += 1
#print(common_words.items())
# sort dict by values descending and slice first 5 to get most common
for k, v in sorted(common_words.items(),
                   key=lambda x: x[1],
                   reverse=True)[:5]:
    print(k ,v)

Now compare this to using `Counter` and its `most_common` method:

In [ ]:
Counter(words).most_common(5)

WOW!

When discovering this my mind was blown and it was a flag to myself that I had to study Python's [awesome standard library](https://docs.python.org/3/library/index.html) more, because it has these Pythonic idioms you can use right out of the box. They will make your code shorter (= less buggy) and more elegant!

Another aha moment was [`deque`](https://pybit.es/collections-deque.html) which we will cover next.

### 4. `deque`

> Deques are a generalization of stacks and queues (the name is pronounced “deck” and is short for “double-ended queue”). Deques support thread-safe, memory efficient appends and pops from either side of the deque with approximately the same O(1) performance in either direction. - [docs](https://docs.python.org/3.7/library/collections.html)




Lists in Python are awesome, probably one of your goto data structure, because they are so widely used and convenient. 

However certain operatings (delete, insert) can get slow when your `list` grows - see [TimeComplexity](https://wiki.python.org/moin/TimeComplexity) for more details. 

If you need to add/remove at both ends of a collection, consider using a `deque`. Let's show this with a practical example using the `timeit` module to measure performance:

First we create two 10 million integers with `range` storing one in a `list ` and the other in a `deque`:

In [ ]:
lst = list(range(10000000))
deq = deque(range(10000000))

Let's do some removing and inserting at random locations in the sequence, a `list` is slow at this because it needs to move all adjacent around ([Grokking Algorithms](https://pybit.es/grokking_algorithms.html) explains this really well). Here is where `deque` is a big win:  

In [ ]:
def insert_and_delete(ds):
    for _ in range(10):
        index = random.choice(range(100))
        ds.remove(index)
        ds.insert(index, index)

%timeit insert_and_delete(lst)

%timeit insert_and_delete(deq)

So when performance matters you really want to explore the alternative data structures in the `collections` module. Another example of a performance win is `ChainMap`:

> A ChainMap groups multiple dicts or other mappings together to create a single, updateable view. If no maps are specified, a single empty dictionary is provided so that a new chain always has at least one mapping. - [docs](https://docs.python.org/3.7/library/collections.html#collections.ChainMap)

## Second day: practice using movie data

For [Code Challenge 13 - Highest Rated Movie Directors](https://pybit.es/codechallenge13.html) we used a nice movie data set. Let's import it here to see some of our newly learned `collections` data types in action.

Let's make a `defaultdict` of movies per directory (keys = directors, values = list of movies). 

In [ ]:
movie_data = 'https://raw.githubusercontent.com/pybites/challenges/master/13/movie_metadata.csv'
movies_csv = 'movies.csv'
urlretrieve(movie_data, movies_csv)

A `namedtuple` is ideal here to describe a movie so we can access movie attributes (e.g. `movie.score`):

In [ ]:
Movie = namedtuple('Movie', 'title year score')

We need some CSV parsing as well here, no worries we got you covered:

In [ ]:
def get_movies_by_director(data=movies_csv):
    """Extracts all movies from csv and stores them in a dictionary
       where keys are directors, and values is a list of movies (named tuples)"""
    directors = defaultdict(list)
    with open(data, encoding='utf-8') as f:
        for line in csv.DictReader(f):
            try:
                director = line['director_name']
                movie = line['movie_title'].replace('\xa0', '')
                year = int(line['title_year'])
                score = float(line['imdb_score'])
            except ValueError:
                continue

            m = Movie(title=movie, year=year, score=score)
            directors[director].append(m)

    return directors

In [ ]:
directors = get_movies_by_director()
# print(directors)

Looking up Christopher Nolan we get all his movies nicely stored in `Movie` `namedtuple` objects.

In [ ]:
directors['Christopher Nolan']

You can do a lot with this data set and [we challenge you to do so](https://pybit.es/codechallenge13.html), but for this example let's just get the 5 directors with the most movies. 

Of course we don't want to re-invent the wheel so we use `Counter`:

In [ ]:
cnt = Counter()
for director, movies in directors.items():
    cnt[director] += len(movies)

cnt.most_common(5)

## Third day: more practice on your own data

We challenge you to find your own data set and try to use the new `collections` data structures yourself. 

Stuck at finding examples? We used `collections` quite a bit for our own [100 Days of Code](https://github.com/pybites/100DaysOfCode/blob/master/LOG.md):

```$ python module_index.py |grep collections
collections        | stdlib | 001, 021, 023, 034, 036, 042, 045, 055, 057, 063, 076, 084, 086, 095, 096```

### Time to share what you've accomplished!

Be sure to share your last couple of days work on Twitter or Facebook. Use the hashtag **#100DaysOfCode**. 

Here are [some examples](https://twitter.com/search?q=%23100DaysOfCode) to inspire you. Consider including [@talkpython](https://twitter.com/talkpython) and [@pybites](https://twitter.com/pybites) in your tweets.

*See a mistake in these instructions? Please [submit a new issue](https://github.com/talkpython/100daysofcode-with-python-course/issues) or fix it and [submit a PR](https://github.com/talkpython/100daysofcode-with-python-course/pulls).*

In [ ]:
Trips = namedtuple('Trips', 'from_station_id to_station_id tripduration')

In [ ]:
def get_bike_by_trip():
    trips = defaultdict(list)
    with open('Divvy_Trips_2016_Q4.csv', encoding='utf-8') as f:
        for line in csv.DictReader(f):
            try:
                bikeid = line['bikeid']
                from_station_id = int(line['from_station_id'])
                to_station_id = int(line['to_station_id'])
                tripduration = int(line['tripduration'])
            except ValueError:
                continue

            t = Trips(from_station_id=from_station_id, to_station_id=to_station_id, tripduration=tripduration)
            trips[bikeid].append(t)

    return trips

In [ ]:
trips = get_bike_by_trip()

In [ ]:
trips['5114']

In [ ]:
# top five most trips
cnt = Counter()
for bikeid, tripdata in trips.items():
    cnt[bikeid] += len(tripduration)

cnt.most_common(5)

In [86]:
# top five longest duration
cnt = Counter()
for bikeid, tripdata in trips.items():
    for i in tripdata:
        # print(bikeid, i.tripduration)
        cnt[bikeid] += i.tripduration
    
cnt.most_common(5)

[('5761', 354043),
 ('5901', 321005),
 ('654', 319795),
 ('1285', 312930),
 ('3909', 295192)]

In [87]:
cars = {
    'Ford': ['Falcon', 'Focus', 'Festiva', 'Fairlane'],
    'Holden': ['Commodore', 'Captiva', 'Barina', 'Trailblazer'],
    'Nissan': ['Maxima', 'Pulsar', '350Z', 'Navara'],
    'Honda': ['Civic', 'Accord', 'Odyssey', 'Jazz'],
    'Jeep': ['Grand Cherokee', 'Cherokee', 'Trailhawk', 'Trackhawk']
}

In [111]:
first = []
for i in cars.keys():
    first.append(cars[i][0])
print(first)

['Falcon', 'Commodore', 'Maxima', 'Civic', 'Grand Cherokee']


In [101]:
mystr = ",".join(cars['Jeep'] )
print(mystr)

Grand Cherokee,Cherokee,Trailhawk,Trackhawk


In [114]:
cars.values()

dict_values([['Falcon', 'Focus', 'Festiva', 'Fairlane'], ['Commodore', 'Captiva', 'Barina', 'Trailblazer'], ['Maxima', 'Pulsar', '350Z', 'Navara'], ['Civic', 'Accord', 'Odyssey', 'Jazz'], ['Grand Cherokee', 'Cherokee', 'Trailhawk', 'Trackhawk']])

In [121]:
for i in cars.values():
    print(i)
l = ['Falcon', 'Focus', 'Festiva', 'Fairlane']

['Falcon', 'Focus', 'Festiva', 'Fairlane']
['Commodore', 'Captiva', 'Barina', 'Trailblazer']
['Maxima', 'Pulsar', '350Z', 'Navara']
['Civic', 'Accord', 'Odyssey', 'Jazz']
['Grand Cherokee', 'Cherokee', 'Trailhawk', 'Trackhawk']


In [119]:
[model for model in l if 'aet' in name]:
    print(model)

SyntaxError: invalid syntax (<ipython-input-119-10f9aedd56e7>, line 1)

In [163]:
l = []
grep = 'CO'
for model in cars.values():
    for i in model:
        # l.append(list(filter(lambda x:'fal' in x.lower(), model)))
        if grep.lower() in i.lower():
            l.append(i)
l.sort()
print(l)

['Accord', 'Commodore', 'Falcon']


In [124]:
for i in l:
    print(i.lower())

falcon
focus
festiva
fairlane


In [126]:
l.lower

AttributeError: 'list' object has no attribute 'lower'

In [164]:
cars = {
    'Ford': ['Falcon', 'Focus', 'Festiva', 'Fairlane'],
    'Holden': ['Commodore', 'Captiva', 'Barina', 'Trailblazer'],
    'Nissan': ['Maxima', 'Pulsar', '350Z', 'Navara'],
    'Honda': ['Civic', 'Accord', 'Odyssey', 'Jazz'],
    'Jeep': ['Grand Cherokee', 'Cherokee', 'Trailhawk', 'Trackhawk']
}

In [165]:
cars.keys()

dict_keys(['Ford', 'Holden', 'Nissan', 'Honda', 'Jeep'])

In [166]:
cars.values()

dict_values([['Falcon', 'Focus', 'Festiva', 'Fairlane'], ['Commodore', 'Captiva', 'Barina', 'Trailblazer'], ['Maxima', 'Pulsar', '350Z', 'Navara'], ['Civic', 'Accord', 'Odyssey', 'Jazz'], ['Grand Cherokee', 'Cherokee', 'Trailhawk', 'Trackhawk']])

In [173]:
for i in cars.keys():
    item = cars[i]
    item.sort()
    cars[i] = item
print(cars)

{'Ford': ['Fairlane', 'Falcon', 'Festiva', 'Focus'], 'Holden': ['Barina', 'Captiva', 'Commodore', 'Trailblazer'], 'Nissan': ['350Z', 'Maxima', 'Navara', 'Pulsar'], 'Honda': ['Accord', 'Civic', 'Jazz', 'Odyssey'], 'Jeep': ['Cherokee', 'Grand Cherokee', 'Trackhawk', 'Trailhawk']}


In [174]:
sum(cars.values(), []) 

['Fairlane',
 'Falcon',
 'Festiva',
 'Focus',
 'Barina',
 'Captiva',
 'Commodore',
 'Trailblazer',
 '350Z',
 'Maxima',
 'Navara',
 'Pulsar',
 'Accord',
 'Civic',
 'Jazz',
 'Odyssey',
 'Cherokee',
 'Grand Cherokee',
 'Trackhawk',
 'Trailhawk']

In [175]:
states = ['Oklahoma', 'Kansas', 'North Carolina', 'Georgia', 'Oregon',
          'Mississippi', 'Minnesota', 'Colorado', 'Alabama',
          'Massachusetts', 'Arizona', 'Connecticut', 'Montana',
          'West Virginia', 'Nebraska', 'New York', 'Nevada', 'Idaho',
          'New Jersey', 'Missouri', 'South Carolina', 'Pennsylvania',
          'Rhode Island', 'New Mexico', 'Alaska', 'New Hampshire',
          'Tennessee', 'Washington', 'Indiana', 'Hawaii', 'Kentucky',
          'Virginia', 'Ohio', 'Wisconsin', 'Maryland', 'Florida',
          'Utah', 'Maine', 'California', 'Vermont', 'Arkansas', 'Wyoming',
          'Louisiana', 'North Dakota', 'South Dakota', 'Texas',
          'Illinois', 'Iowa', 'Michigan', 'Delaware']

In [193]:
n = 10
i = n - 1
st_list = []
while i <= len(states):
    st_list.append(states[i])
    i += n
print(st_list)

['Massachusetts', 'Missouri', 'Hawaii', 'Vermont', 'Delaware']


In [195]:
us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
                   'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
                   'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL',
                   'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
                   'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
                   'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
                   'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
                   'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
                   'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE',
                   'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
                   'New Mexico': 'NM', 'New York': 'NY',
                   'North Carolina': 'NC', 'North Dakota': 'ND',
                   'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
                   'Pennsylvania': 'PA', 'Rhode Island': 'RI',
                   'South Carolina': 'SC', 'South Dakota': 'SD',
                   'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
                   'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
                   'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [196]:
us_state_abbrev['Alabama']

'AL'

In [200]:
us_state_abbrev.get('Alabama', 'foo')

'AL'

In [202]:
print(max(states, key=len))

North Carolina


In [228]:
abbr = list(us_state_abbrev.values())[:10]
abbr.sort()
abbr

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA']

In [229]:
states.sort()
last_states = states[-10:]
last_states

['South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [231]:
abbr + last_states 

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DE',
 'FL',
 'GA',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']